# 🚀 CYCLE-TO-PEAK PERFORMANCE PREDICTOR
## Implementation of the Three-Pillar Approach

This notebook implements:
- **Pillar 1**: Training Cycle Calculation (J-days)
- **Pillar 2**: Advanced Feature Engineering (GPS + Wyscout)
- **Pillar 3**: AI-Powered Performance Prediction (LSTM)

Each pillar will be tested before proceeding to the next.

---
# PILLAR 1: Training Cycle Calculation
**Goal**: Map each training session to its J-day (days until next match)

In [27]:
# ============================================================
# PILLAR 1: TRAINING CYCLE CALCULATION
# ============================================================
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import timedelta

print("="*60)
print("PILLAR 1: Training Cycle Calculation")
print("="*60)

# Load data
DATA_DIR = Path("hackathon_Data/hackathon/Data After Extraction/CSV")

def safe_read(path, encoding='utf-8'):
    for enc in [encoding, 'latin1', 'cp1252']:
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except:
            continue
    return pd.read_csv(path, low_memory=False)

# Read GPS data
gps_match = safe_read(DATA_DIR / "matchs_gps.csv")
gps_train = safe_read(DATA_DIR / "training_gps.csv")

# Normalize column names
def normalize_cols(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
    return df

gps_match = normalize_cols(gps_match)
gps_train = normalize_cols(gps_train)

# Parse dates
gps_match['date'] = pd.to_datetime(gps_match['date'], errors='coerce')
gps_train['date'] = pd.to_datetime(gps_train['date'], errors='coerce')

print(f"\n✓ Loaded GPS Match data: {gps_match.shape}")
print(f"✓ Loaded GPS Training data: {gps_train.shape}")
print(f"\n✓ Date range (Match): {gps_match['date'].min()} to {gps_match['date'].max()}")
print(f"✓ Date range (Training): {gps_train['date'].min()} to {gps_train['date'].max()}")

PILLAR 1: Training Cycle Calculation

✓ Loaded GPS Match data: (2108, 812)
✓ Loaded GPS Training data: (7903, 810)

✓ Date range (Match): 2025-07-30 00:00:00 to 2025-11-06 00:00:00
✓ Date range (Training): 2025-07-15 00:00:00 to 2025-11-05 00:00:00


In [28]:
# Step 1.1: Create unified GPS calendar with event types
gps_match['event_type'] = 'Match'
gps_train['event_type'] = 'Training'

# Combine both datasets
gps_all = pd.concat([gps_match, gps_train], ignore_index=True)

# Create base calendar
calendar = gps_all[['name', 'team_name', 'date', 'event_type']].copy()
calendar.columns = ['player_name', 'team_name', 'date', 'event_type']
calendar = calendar.dropna(subset=['player_name', 'date'])
calendar = calendar.drop_duplicates()
calendar = calendar.sort_values(['player_name', 'date']).reset_index(drop=True)

print(f"\n✓ Calendar created: {calendar.shape}")
print(f"✓ Unique players: {calendar['player_name'].nunique()}")
print(f"✓ Event type distribution:")
print(calendar['event_type'].value_counts())

# Show sample
print("\n📋 Sample calendar:")
print(calendar.head(15))


✓ Calendar created: (2041, 4)
✓ Unique players: 42
✓ Event type distribution:
event_type
Training    1717
Match        324
Name: count, dtype: int64

📋 Sample calendar:
              player_name team_name       date event_type
0   Abdelmounaim Boutouil  AL HAZEM 2025-09-07   Training
1   Abdelmounaim Boutouil  AL HAZEM 2025-09-08   Training
2   Abdelmounaim Boutouil  AL HAZEM 2025-09-09   Training
3   Abdelmounaim Boutouil  AL HAZEM 2025-09-10   Training
4   Abdelmounaim Boutouil  AL HAZEM 2025-09-14   Training
5   Abdelmounaim Boutouil  AL HAZEM 2025-09-15   Training
6   Abdelmounaim Boutouil  AL HAZEM 2025-09-16   Training
7   Abdelmounaim Boutouil  AL HAZEM 2025-09-17   Training
8   Abdelmounaim Boutouil  AL HAZEM 2025-09-18   Training
9   Abdelmounaim Boutouil  AL HAZEM 2025-09-19   Training
10  Abdelmounaim Boutouil  AL HAZEM 2025-09-21   Training
11  Abdelmounaim Boutouil  AL HAZEM 2025-09-22   Training
12  Abdelmounaim Boutouil  AL HAZEM 2025-09-24   Training
13  Abdelmounaim B

In [29]:
# Step 1.2: Calculate J-days (Training Cycle)
def calculate_j_days(df):
    """
    For each player, calculate:
    - next_match_date: date of the next match
    - days_to_next_match: days until next match
    - J_label: J-Day, J-1, J-2, etc.
    """
    df = df.sort_values(['player_name', 'date']).copy()
    results = []
    
    for player, grp in df.groupby('player_name'):
        grp = grp.sort_values('date').reset_index(drop=True)
        
        # Find match dates for this player
        match_dates = grp[grp['event_type'] == 'Match']['date'].values
        
        # For each row, find the next match
        next_match = []
        for date in grp['date']:
            future_matches = match_dates[match_dates > date]
            if len(future_matches) > 0:
                next_match.append(pd.Timestamp(future_matches[0]))
            else:
                next_match.append(pd.NaT)
        
        grp['next_match_date'] = next_match
        
        # Calculate days to next match
        grp['days_to_next_match'] = (grp['next_match_date'] - grp['date']).dt.days
        
        # Create J-label
        def make_j_label(row):
            if pd.isna(row['next_match_date']):
                return None
            if row['event_type'] == 'Match':
                return 'J-Day'
            if row['days_to_next_match'] > 0:
                return f"J-{int(row['days_to_next_match'])}"
            return None
        
        grp['J_label'] = grp.apply(make_j_label, axis=1)
        results.append(grp)
    
    return pd.concat(results, ignore_index=True)

# Apply J-day calculation
calendar = calculate_j_days(calendar)

print("\n✓ J-days calculated!")
print(f"\n📊 J-label distribution:")
print(calendar['J_label'].value_counts().sort_index())

# Show examples
print("\n📋 Example training cycle for one player:")
sample_player = calendar['player_name'].iloc[0]
sample = calendar[calendar['player_name'] == sample_player].head(20)
print(sample[['player_name', 'date', 'event_type', 'J_label', 'days_to_next_match']])


✓ J-days calculated!

📊 J-label distribution:
J_label
J-1      309
J-10      14
J-11      33
J-12      39
J-13      38
J-14      20
J-15      26
J-16      15
J-17       5
J-18       5
J-19       6
J-2      312
J-20       6
J-21       4
J-22       1
J-23       1
J-27       1
J-28       2
J-29       1
J-3      212
J-30       1
J-31       2
J-32       1
J-33       2
J-34       2
J-35       2
J-36       2
J-37       1
J-4      176
J-5      137
J-6       55
J-7       25
J-8       36
J-9       31
J-Day    291
Name: count, dtype: int64

📋 Example training cycle for one player:
              player_name       date event_type J_label  days_to_next_match
0   Abdelmounaim Boutouil 2025-09-07   Training    J-30                30.0
1   Abdelmounaim Boutouil 2025-09-08   Training    J-29                29.0
2   Abdelmounaim Boutouil 2025-09-09   Training    J-28                28.0
3   Abdelmounaim Boutouil 2025-09-10   Training    J-27                27.0
4   Abdelmounaim Boutouil 2025-09-14   Tra

In [30]:
# Step 1.3: Validate Pillar 1
print("\n" + "="*60)
print("PILLAR 1 VALIDATION")
print("="*60)

# Check 1: Are there training days with J-labels?
training_with_j = calendar[(calendar['event_type'] == 'Training') & (calendar['J_label'].notna())]
print(f"\n✓ Training sessions with J-labels: {len(training_with_j)} / {len(calendar[calendar['event_type'] == 'Training'])}")

# Check 2: Are match days labeled as J-Day?
match_j_day = calendar[(calendar['event_type'] == 'Match') & (calendar['J_label'] == 'J-Day')]
print(f"✓ Match days labeled as J-Day: {len(match_j_day)} / {len(calendar[calendar['event_type'] == 'Match'])}")

# Check 3: Show typical training cycle
print("\n📋 Typical training cycle (J-6 to J-Day):")
cycle_example = calendar[calendar['J_label'].isin(['J-6', 'J-5', 'J-4', 'J-3', 'J-2', 'J-1', 'J-Day'])].head(10)
print(cycle_example[['player_name', 'date', 'event_type', 'J_label']])

# Save Pillar 1 output
OUT_DIR = Path("data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)
calendar.to_csv(OUT_DIR / "pillar1_calendar_j_days.csv", index=False)
print(f"\n💾 Saved: {OUT_DIR / 'pillar1_calendar_j_days.csv'}")

print("\n✅ PILLAR 1 COMPLETE - Ready for Pillar 2")


PILLAR 1 VALIDATION

✓ Training sessions with J-labels: 1523 / 1717
✓ Match days labeled as J-Day: 291 / 324

📋 Typical training cycle (J-6 to J-Day):
              player_name       date event_type J_label
14  Abdelmounaim Boutouil 2025-10-02   Training     J-5
15  Abdelmounaim Boutouil 2025-10-03   Training     J-4
16  Abdelmounaim Boutouil 2025-10-04   Training     J-3
17  Abdelmounaim Boutouil 2025-10-05   Training     J-2
18  Abdelmounaim Boutouil 2025-10-06   Training     J-1
19  Abdelmounaim Boutouil 2025-10-07      Match   J-Day
26  Abdelmounaim Boutouil 2025-10-22   Training     J-3
27  Abdelmounaim Boutouil 2025-10-23   Training     J-2
28  Abdelmounaim Boutouil 2025-10-24   Training     J-1
29  Abdelmounaim Boutouil 2025-10-25      Match   J-Day

💾 Saved: data\processed\pillar1_calendar_j_days.csv

✅ PILLAR 1 COMPLETE - Ready for Pillar 2


---
# PILLAR 2: Advanced Feature Engineering
**Goal**: Extract physical (GPS) and technical (Wyscout) features for each player-date

In [31]:
# ============================================================
# PILLAR 2: FEATURE ENGINEERING - GPS Physical Metrics
# ============================================================

print("\n" + "="*60)
print("PILLAR 2: Feature Engineering")
print("="*60)

# Step 2.1: Extract GPS Physical Features
print("\n[Step 2.1] Extracting GPS physical metrics...")

# Key GPS metrics based on the PDF methodology
gps_features = {
    # Volume metrics
    'total_distance_(m)': 'total_distance',
    'total_duration': 'total_duration',
    
    # Intensity metrics
    'hs_distance_(m)': 'high_speed_distance',  # >19.8 km/h
    'sprint': 'sprint_distance',  # >25.2 km/h
    'distancia_>_20': 'distance_above_20',
    'distancia_>_25': 'distance_above_25',
    
    # Player Load (accelerometer)
    'total_player_load': 'total_player_load',
    'player_load_per_minute': 'player_load_per_min',
    'average_player_load': 'avg_player_load',
    
    # Explosiveness metrics
    'acceleration_b3_efforts_(gen_2)': 'high_accel_efforts',
    'deceleration_b3_efforts_(gen_2)': 'high_decel_efforts',
    'accel_+_decel_efforts': 'total_accel_decel',
    'explosive_efforts': 'explosive_efforts',
    
    # Metabolic power
    'high_metabolic_load_distance_(m)': 'high_metabolic_distance',
    'equivalent_distance_(m)': 'equivalent_distance',
    
    # Heart rate
    'avg_heart_rate': 'avg_hr',
    'maximum_heart_rate': 'max_hr',
    'heart_rate_exertion': 'hr_exertion',
}

# Extract features from gps_all
def extract_gps_features(gps_df, feature_map):
    """Extract and rename GPS features"""
    available_features = {}
    for old_col, new_col in feature_map.items():
        if old_col in gps_df.columns:
            available_features[old_col] = new_col
    
    # Select columns
    id_cols = ['name', 'team_name', 'date', 'event_type']
    feature_cols = list(available_features.keys())
    
    df = gps_df[id_cols + feature_cols].copy()
    df.rename(columns={'name': 'player_name'}, inplace=True)
    df.rename(columns=available_features, inplace=True)
    
    return df

gps_features_df = extract_gps_features(gps_all, gps_features)

print(f"✓ Extracted {len(gps_features)} GPS features")
print(f"✓ GPS features shape: {gps_features_df.shape}")
print(f"\n📊 Available GPS metrics:")
for col in gps_features_df.columns:
    if col not in ['player_name', 'team_name', 'date', 'event_type']:
        print(f"  - {col}")
        
# Show sample
print("\n📋 Sample GPS features:")
print(gps_features_df.head(3))


PILLAR 2: Feature Engineering

[Step 2.1] Extracting GPS physical metrics...
✓ Extracted 18 GPS features
✓ GPS features shape: (10011, 22)

📊 Available GPS metrics:
  - total_distance
  - total_duration
  - high_speed_distance
  - sprint_distance
  - distance_above_20
  - distance_above_25
  - total_player_load
  - player_load_per_min
  - avg_player_load
  - high_accel_efforts
  - high_decel_efforts
  - total_accel_decel
  - explosive_efforts
  - high_metabolic_distance
  - equivalent_distance
  - avg_hr
  - max_hr
  - hr_exertion

📋 Sample GPS features:
            player_name team_name       date event_type  total_distance  \
0    Yousef Al-Shammari  AL HAZEM 2025-08-08      Match      1410.80823   
1  Abdullah Al-Shanqiti  AL HAZEM 2025-09-23      Match      1234.82007   
2    Yousef Al-Shammari  AL HAZEM 2025-08-08      Match      7038.72546   

  total_duration  high_speed_distance  sprint_distance  distance_above_20  \
0       00:15:00                 0.00                0      

In [32]:
# Step 2.2: Extract Wyscout Technical Features
print("\n[Step 2.2] Extracting Wyscout technical metrics...")

# Load Wyscout data
wy_out = safe_read(DATA_DIR / "wyscout_players_outfield.csv")
wy_gk = safe_read(DATA_DIR / "wyscout_players_goalkeeper.csv")

wy_out = normalize_cols(wy_out)
wy_gk = normalize_cols(wy_gk)

# Parse dates
wy_out['date'] = pd.to_datetime(wy_out['date'], errors='coerce')
wy_gk['date'] = pd.to_datetime(wy_gk['date'], errors='coerce')

# Position-specific KPIs from the PDF
outfield_features = {
    # Attack
    'attack_buts': 'goals',
    'attack_tirs_total': 'shots',
    'attack_tirs_cadres': 'shots_on_target',
    'attack_xg': 'xg',
    'attack_passes_decisives': 'assists',
    'attack_dribbles_reussis': 'successful_dribbles',
    'attack_dribbles_total': 'total_dribbles',
    'attack_duels_offensifs_gagnes': 'offensive_duels_won',
    'attack_duels_offensifs_total': 'offensive_duels_total',
    
    # Passing
    'general_passes_precises': 'accurate_passes',
    'general_passes_total': 'total_passes',
    'pass_passes_avant_precises': 'forward_passes_accurate',
    'pass_passes_avant_total': 'forward_passes_total',
    'pass_passes_tiers3_precises': 'final_third_passes_accurate',
    'pass_passes_tiers3_total': 'final_third_passes_total',
    
    # Defense
    'defense_duels_defensifs_gagnes': 'defensive_duels_won',
    'defense_duels_defensifs_total': 'defensive_duels_total',
    'defense_interceptions_total': 'interceptions',
    'defense_duels_aeriens_gagnes': 'aerial_duels_won',
    'defense_duels_aeriens_total': 'aerial_duels_total',
}

gk_features = {
    'goalkeeper_buts_concedes': 'goals_conceded',
    'goalkeeper_tirs_contre_total': 'shots_against',
    'goalkeeper_tirs_contre_cadres': 'shots_on_target_against',
    'goalkeeper_xcg': 'xcg',
    'goalkeeper_sorties_total': 'exits',
}

# Extract outfield features
def extract_wyscout_features(wy_df, feature_map, position_group):
    """Extract and rename Wyscout features"""
    available_features = {}
    for old_col, new_col in feature_map.items():
        if old_col in wy_df.columns:
            available_features[old_col] = new_col
    
    id_cols = ['player', 'team_name', 'date', 'match', 'competition']
    feature_cols = list(available_features.keys())
    
    df = wy_df[id_cols + feature_cols].copy()
    df.rename(columns={'player': 'player_name'}, inplace=True)
    df.rename(columns=available_features, inplace=True)
    df['position_group'] = position_group
    
    return df

wy_out_features = extract_wyscout_features(wy_out, outfield_features, 'Outfield')
wy_gk_features = extract_wyscout_features(wy_gk, gk_features, 'GK')

# Combine
wyscout_features_df = pd.concat([wy_out_features, wy_gk_features], ignore_index=True)

print(f"✓ Extracted Wyscout features for {wy_out_features.shape[0]} outfield + {wy_gk_features.shape[0]} GK records")
print(f"✓ Wyscout features shape: {wyscout_features_df.shape}")
print(f"\n📊 Sample Wyscout metrics:")
print(wyscout_features_df.head(3))


[Step 2.2] Extracting Wyscout technical metrics...
✓ Extracted Wyscout features for 141558 outfield + 15323 GK records
✓ Wyscout features shape: (156881, 31)

📊 Sample Wyscout metrics:
          player_name team_name       date                        match  \
0        Roger Ibañez   AL AHLI 2021-03-18  Shakhtar Donetsk - Roma 1:2   
1         Alaa  Hejji      NEOM 2023-04-24      Al Nassr - Al Wahda 0:1   
2  Yousef Al Shammari  AL HAZEM 2021-12-20       Al Batin - Al Hazm 1:0   

                  competition  goals  shots  shots_on_target    xg  assists  \
0  Europe. UEFA Europa League    0.0    0.0              0.0  0.00      0.0   
1    Saudi Arabia. King's Cup    0.0    0.0              0.0  0.00      0.0   
2    Saudi Arabia. King's Cup    0.0    1.0              0.0  0.03      0.0   

   ...  defensive_duels_total  interceptions  aerial_duels_won  \
0  ...                   12.0            4.0               1.0   
1  ...                    1.0            0.0               0.0  

In [33]:
# Step 2.3: Merge Calendar + GPS + Wyscout
print("\n[Step 2.3] Merging all features...")

# Start with calendar (has J-days)
master_df = calendar.copy()

# Merge GPS features
master_df = master_df.merge(
    gps_features_df,
    on=['player_name', 'team_name', 'date', 'event_type'],
    how='left'
)

# Merge Wyscout features (only for match days)
master_df = master_df.merge(
    wyscout_features_df,
    on=['player_name', 'team_name', 'date'],
    how='left'
)

print(f"✓ Master dataset created: {master_df.shape}")
print(f"✓ Columns: {list(master_df.columns)}")

# Calculate derived metrics
print("\n[Step 2.4] Calculating derived metrics...")

# Passing accuracy %
if 'accurate_passes' in master_df.columns and 'total_passes' in master_df.columns:
    master_df['passing_accuracy_pct'] = (master_df['accurate_passes'] / master_df['total_passes'] * 100).round(2)

# Dribble success %
if 'successful_dribbles' in master_df.columns and 'total_dribbles' in master_df.columns:
    master_df['dribble_success_pct'] = (master_df['successful_dribbles'] / master_df['total_dribbles'] * 100).round(2)

# Defensive duel win %
if 'defensive_duels_won' in master_df.columns and 'defensive_duels_total' in master_df.columns:
    master_df['defensive_duels_won_pct'] = (master_df['defensive_duels_won'] / master_df['defensive_duels_total'] * 100).round(2)

# Offensive duel win %
if 'offensive_duels_won' in master_df.columns and 'offensive_duels_total' in master_df.columns:
    master_df['offensive_duels_won_pct'] = (master_df['offensive_duels_won'] / master_df['offensive_duels_total'] * 100).round(2)

print(f"✓ Added derived percentage metrics")

# Show summary
print("\n📊 Feature summary:")
print(f"  - Physical (GPS) features: ~{len([c for c in master_df.columns if c in gps_features.values()])}")
print(f"  - Technical (Wyscout) features: ~{len([c for c in master_df.columns if c in list(outfield_features.values()) + list(gk_features.values())])}")
print(f"  - Derived metrics: 4")
print(f"  - Total features: {master_df.shape[1]}")

print("\n📋 Sample master dataset:")
print(master_df.head(3))


[Step 2.3] Merging all features...
✓ Master dataset created: (12251, 53)
✓ Columns: ['player_name', 'team_name', 'date', 'event_type', 'next_match_date', 'days_to_next_match', 'J_label', 'total_distance', 'total_duration', 'high_speed_distance', 'sprint_distance', 'distance_above_20', 'distance_above_25', 'total_player_load', 'player_load_per_min', 'avg_player_load', 'high_accel_efforts', 'high_decel_efforts', 'total_accel_decel', 'explosive_efforts', 'high_metabolic_distance', 'equivalent_distance', 'avg_hr', 'max_hr', 'hr_exertion', 'match', 'competition', 'goals', 'shots', 'shots_on_target', 'xg', 'assists', 'successful_dribbles', 'total_dribbles', 'offensive_duels_won', 'offensive_duels_total', 'accurate_passes', 'total_passes', 'forward_passes_accurate', 'forward_passes_total', 'final_third_passes_accurate', 'final_third_passes_total', 'defensive_duels_won', 'defensive_duels_total', 'interceptions', 'aerial_duels_won', 'aerial_duels_total', 'position_group', 'goals_conceded', 'sh

In [34]:
# Step 2.5: Validate Pillar 2
print("\n" + "="*60)
print("PILLAR 2 VALIDATION")
print("="*60)

# Check 1: Feature coverage
match_rows = master_df[master_df['event_type'] == 'Match']
training_rows = master_df[master_df['event_type'] == 'Training']

print(f"\n✓ Match rows with GPS data: {match_rows['total_distance'].notna().sum()} / {len(match_rows)}")
print(f"✓ Match rows with Wyscout data: {match_rows['goals'].notna().sum() if 'goals' in match_rows.columns else 0} / {len(match_rows)}")
print(f"✓ Training rows with GPS data: {training_rows['total_distance'].notna().sum()} / {len(training_rows)}")

# Check 2: Missing values analysis
print("\n📊 Missing values in key features:")
key_features = ['total_distance', 'high_speed_distance', 'total_player_load', 'goals', 'shots', 'accurate_passes']
for feat in key_features:
    if feat in master_df.columns:
        missing_pct = (master_df[feat].isna().sum() / len(master_df) * 100)
        print(f"  {feat:25s}: {missing_pct:.1f}% missing")

# Check 3: Data quality
print("\n📊 Data quality checks:")
print(f"  Total rows: {len(master_df)}")
print(f"  Rows with J-label: {master_df['J_label'].notna().sum()}")
print(f"  Rows with GPS data: {master_df['total_distance'].notna().sum()}")
print(f"  Date range: {master_df['date'].min()} to {master_df['date'].max()}")

# Save Pillar 2 output
master_df.to_csv(OUT_DIR / "pillar2_master_features.csv", index=False)
print(f"\n💾 Saved: {OUT_DIR / 'pillar2_master_features.csv'}")

print("\n✅ PILLAR 2 COMPLETE - Ready for Pillar 3")


PILLAR 2 VALIDATION

✓ Match rows with GPS data: 4348 / 4348
✓ Match rows with Wyscout data: 2536 / 4348
✓ Training rows with GPS data: 7903 / 7903

📊 Missing values in key features:
  total_distance           : 0.0% missing
  high_speed_distance      : 0.0% missing
  total_player_load        : 0.0% missing
  goals                    : 79.3% missing
  shots                    : 79.3% missing
  accurate_passes          : 79.3% missing

📊 Data quality checks:
  Total rows: 12251
  Rows with J-label: 11360
  Rows with GPS data: 12251
  Date range: 2025-07-15 00:00:00 to 2025-11-06 00:00:00

💾 Saved: data\processed\pillar2_master_features.csv

✅ PILLAR 2 COMPLETE - Ready for Pillar 3


---
# PILLAR 3: AI-Powered Performance Prediction
**Goal**: 
1. Generate historical Match Performance Rating (1-10)
2. Train LSTM to predict next match rating based on training cycle

In [35]:
# ============================================================
# PILLAR 3.1: Generate Historical Match Performance Rating
# ============================================================

print("\n" + "="*60)
print("PILLAR 3: AI-Powered Performance Prediction")
print("="*60)

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

print("\n[Step 3.1] Generating Match Performance Ratings...")

# Filter match days only
match_data = master_df[master_df['event_type'] == 'Match'].copy()

print(f"✓ Match records: {len(match_data)}")

# Define feature groups for rating calculation
gps_rating_features = [
    'total_distance', 'high_speed_distance', 'sprint_distance',
    'total_player_load', 'high_accel_efforts', 'high_decel_efforts'
]

wyscout_rating_features = [
    'goals', 'shots', 'xg', 'assists', 'successful_dribbles',
    'accurate_passes', 'total_passes', 'offensive_duels_won',
    'defensive_duels_won', 'interceptions'
]

# Filter available features
available_gps = [f for f in gps_rating_features if f in match_data.columns]
available_wyscout = [f for f in wyscout_rating_features if f in match_data.columns]

print(f"\n📊 Features for rating:")
print(f"  GPS features: {len(available_gps)}")
print(f"  Wyscout features: {len(available_wyscout)}")

# Create rating dataset (only matches with sufficient data)
rating_data = match_data.dropna(subset=available_gps + available_wyscout[:3])  # At least some key Wyscout features

print(f"✓ Matches with sufficient data for rating: {len(rating_data)}")

if len(rating_data) > 0:
    # Combine features
    all_features = available_gps + available_wyscout
    X = rating_data[all_features].fillna(0)
    
    # Standardize (Z-scores)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Apply PCA to determine feature weights
    pca = PCA(n_components=min(5, len(all_features)))
    pca.fit(X_scaled)
    
    # Use first principal component as basis for weights
    weights = np.abs(pca.components_[0])
    weights = weights / weights.sum()  # Normalize
    
    # Calculate weighted score
    weighted_scores = (X_scaled * weights).sum(axis=1)
    
    # Normalize to 1-10 scale
    min_score, max_score = weighted_scores.min(), weighted_scores.max()
    rating_data['match_performance_rating'] = 1 + 9 * (weighted_scores - min_score) / (max_score - min_score)
    rating_data['match_performance_rating'] = rating_data['match_performance_rating'].round(2)
    
    print(f"\n✓ Performance ratings generated!")
    print(f"\n📊 Rating statistics:")
    print(rating_data['match_performance_rating'].describe())
    
    # Show examples
    print("\n📋 Sample match ratings:")
    sample_cols = ['player_name', 'date', 'match_performance_rating', 'goals', 'shots', 'total_distance']
    print(rating_data[sample_cols].head(10))
else:
    print("⚠️ Not enough data to generate ratings")


PILLAR 3: AI-Powered Performance Prediction

[Step 3.1] Generating Match Performance Ratings...
✓ Match records: 4348

📊 Features for rating:
  GPS features: 6
  Wyscout features: 10
✓ Matches with sufficient data for rating: 2536

✓ Performance ratings generated!

📊 Rating statistics:
count    2536.000000
mean        2.218375
std         0.952515
min         1.000000
25%         1.550000
50%         1.940000
75%         2.660000
max        10.000000
Name: match_performance_rating, dtype: float64

📋 Sample match ratings:
        player_name       date  match_performance_rating  goals  shots  \
2768  Aboubacar Bah 2025-08-28                      1.91    0.0    0.0   
2769  Aboubacar Bah 2025-08-28                      2.02    0.0    0.0   
2770  Aboubacar Bah 2025-08-28                      2.08    0.0    0.0   
2771  Aboubacar Bah 2025-08-28                      3.29    0.0    1.0   
2772  Aboubacar Bah 2025-08-28                      2.12    0.0    1.0   
2773  Aboubacar Bah 2025-08-

C:\Users\pc\AppData\Local\Temp\ipykernel_4344\1601541816.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_data['match_performance_rating'] = 1 + 9 * (weighted_scores - min_score) / (max_score - min_score)
C:\Users\pc\AppData\Local\Temp\ipykernel_4344\1601541816.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_data['match_performance_rating'] = rating_data['match_performance_rating'].round(2)


In [36]:
# ============================================================
# PILLAR 3.2: Prepare Time-Series Data for LSTM
# ============================================================

print("\n[Step 3.2] Preparing time-series sequences for LSTM...")

# Merge ratings back to master dataset
master_df = master_df.merge(
    rating_data[['player_name', 'date', 'match_performance_rating']],
    on=['player_name', 'date'],
    how='left'
)

# For each player, we need sequences of training days leading to a match
def create_sequences(df, sequence_length=7):
    """
    Create sequences of training data leading up to matches
    
    For each match, look back `sequence_length` days and collect:
    - Daily GPS metrics (training load)
    - Last match performance rating
    - J-label context
    
    Target: Next match performance rating
    """
    sequences = []
    targets = []
    metadata = []
    
    for player in df['player_name'].unique():
        player_data = df[df['player_name'] == player].sort_values('date').reset_index(drop=True)
        
        # Find matches with ratings
        match_indices = player_data[
            (player_data['event_type'] == 'Match') & 
            (player_data['match_performance_rating'].notna())
        ].index
        
        for match_idx in match_indices:
            match_row = player_data.iloc[match_idx]
            match_rating = match_row['match_performance_rating']
            
            # Look back for training data
            lookback_start = max(0, match_idx - sequence_length)
            sequence_data = player_data.iloc[lookback_start:match_idx]
            
            if len(sequence_data) > 0:
                # Extract features for sequence
                seq_features = []
                for _, row in sequence_data.iterrows():
                    day_features = [
                        row.get('total_distance', 0) or 0,
                        row.get('high_speed_distance', 0) or 0,
                        row.get('total_player_load', 0) or 0,
                        row.get('high_accel_efforts', 0) or 0,
                        1 if row['event_type'] == 'Match' else 0,  # Is match day
                        row.get('days_to_next_match', 0) or 0,
                    ]
                    seq_features.append(day_features)
                
                # Pad if needed
                while len(seq_features) < sequence_length:
                    seq_features.insert(0, [0] * len(seq_features[0]))
                
                sequences.append(seq_features[:sequence_length])
                targets.append(match_rating)
                metadata.append({
                    'player': player,
                    'date': match_row['date'],
                    'team': match_row.get('team_name', ''),
                })
    
    return np.array(sequences), np.array(targets), metadata

# Create sequences
SEQ_LENGTH = 10  # Look back 10 days before each match
X_seq, y_seq, seq_meta = create_sequences(master_df, sequence_length=SEQ_LENGTH)

print(f"\n✓ Created {len(X_seq)} sequences")
print(f"✓ Sequence shape: {X_seq.shape}")
print(f"  - Samples: {X_seq.shape[0]}")
print(f"  - Time steps: {X_seq.shape[1]}")
print(f"  - Features per step: {X_seq.shape[2]}")
print(f"✓ Target shape: {y_seq.shape}")

print("\n📊 Target (rating) distribution:")
print(f"  Mean: {y_seq.mean():.2f}")
print(f"  Std: {y_seq.std():.2f}")
print(f"  Min: {y_seq.min():.2f}")
print(f"  Max: {y_seq.max():.2f}")


[Step 3.2] Preparing time-series sequences for LSTM...

✓ Created 212317 sequences
✓ Sequence shape: (212317, 10, 6)
  - Samples: 212317
  - Time steps: 10
  - Features per step: 6
✓ Target shape: (212317,)

📊 Target (rating) distribution:
  Mean: 2.21
  Std: 0.95
  Min: 1.00
  Max: 10.00


In [37]:
# ============================================================
# PILLAR 3.3: Build and Train LSTM Model
# ============================================================

print("\n[Step 3.3] Building LSTM model...")

# Check if we have enough data
if len(X_seq) < 10:
    print("⚠️ Not enough sequences for training (need at least 10)")
    print("   This is likely due to limited match data with ratings.")
    print("   The pipeline is ready - just need more data!")
else:
    from sklearn.model_selection import train_test_split
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_seq, y_seq, test_size=0.2, random_state=42
    )
    
    print(f"✓ Training set: {X_train.shape[0]} samples")
    print(f"✓ Test set: {X_test.shape[0]} samples")
    
    # Normalize features
    from sklearn.preprocessing import MinMaxScaler
    
    # Reshape for scaling
    n_samples, n_steps, n_features = X_train.shape
    X_train_reshaped = X_train.reshape(-1, n_features)
    X_test_reshaped = X_test.reshape(-1, n_features)
    
    scaler_X = MinMaxScaler()
    X_train_scaled = scaler_X.fit_transform(X_train_reshaped).reshape(n_samples, n_steps, n_features)
    X_test_scaled = scaler_X.transform(X_test_reshaped).reshape(X_test.shape[0], n_steps, n_features)
    
    print("\n✓ Features normalized")
    
    # Build LSTM model
    print("\n[Building LSTM Architecture...]")
    
    try:
        import tensorflow as tf
        from tensorflow import keras
        from tensorflow.keras import layers
        
        model = keras.Sequential([
            layers.LSTM(64, activation='relu', return_sequences=True, input_shape=(SEQ_LENGTH, n_features)),
            layers.Dropout(0.2),
            layers.LSTM(32, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(16, activation='relu'),
            layers.Dense(1)  # Output: predicted rating
        ])
        
        model.compile(
            optimizer='adam',
            loss='mse',
            metrics=['mae']
        )
        
        print("✓ LSTM model built!")
        print("\n📋 Model architecture:")
        model.summary()
        
        # Train model
        print("\n[Training LSTM...]")
        history = model.fit(
            X_train_scaled, y_train,
            validation_split=0.2,
            epochs=50,
            batch_size=16,
            verbose=0
        )
        
        print("✓ Training complete!")
        
        # Evaluate
        test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=0)
        print(f"\n📊 Test Performance:")
        print(f"  Loss (MSE): {test_loss:.4f}")
        print(f"  MAE: {test_mae:.4f}")
        
        # Make predictions
        y_pred = model.predict(X_test_scaled, verbose=0).flatten()
        
        print("\n📋 Sample predictions:")
        for i in range(min(5, len(y_test))):
            print(f"  Actual: {y_test[i]:.2f} | Predicted: {y_pred[i]:.2f}")
        
        # Save model
        model.save(OUT_DIR / "lstm_performance_predictor.h5")
        print(f"\n💾 Model saved: {OUT_DIR / 'lstm_performance_predictor.h5'}")
        
    except ImportError:
        print("⚠️ TensorFlow not installed. Install with: pip install tensorflow")
        print("   The data preparation is complete - just need TensorFlow to train the model!")
    
print("\n✅ PILLAR 3 COMPLETE!")


[Step 3.3] Building LSTM model...
✓ Training set: 169853 samples
✓ Test set: 42464 samples

✓ Features normalized

[Building LSTM Architecture...]


c:\Users\pc\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✓ LSTM model built!

📋 Model architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10, 64)         │        18,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,137 (121.63 KB)

 Trainable params: 31,137 (121.63 KB)

 Non-trainable params: 0 (0.00 B)


[Training LSTM...]
✓ Training complete!

📊 Test Performance:
  Loss (MSE): 0.7868
  MAE: 0.6668



📋 Sample predictions:
  Actual: 2.04 | Predicted: 2.12
  Actual: 2.73 | Predicted: 2.16
  Actual: 1.31 | Predicted: 2.16
  Actual: 1.53 | Predicted: 2.31
  Actual: 1.24 | Predicted: 2.14

💾 Model saved: data\processed\lstm_performance_predictor.h5

✅ PILLAR 3 COMPLETE!


In [1]:
# ============================================================
# FINAL VALIDATION & SUMMARY
# ============================================================

print("\n" + "="*60)
print("🎯 CYCLE-TO-PEAK PIPELINE SUMMARY")
print("="*60)

print("\n✅ PILLAR 1: Training Cycle Calculation")
print(f"  • Created calendar with J-day labels")
print(f"  • Total records: {len(calendar)}")
print(f"  • Players tracked: {calendar['player_name'].nunique()}")
print(f"  • Date range: {calendar['date'].min().date()} to {calendar['date'].max().date()}")

print("\n✅ PILLAR 2: Feature Engineering")
print(f"  • Master dataset: {master_df.shape[0]} rows × {master_df.shape[1]} columns")
print(f"  • GPS physical metrics: {len(available_gps)}")
print(f"  • Wyscout technical metrics: {len(available_wyscout)}")
print(f"  • Match records: {len(master_df[master_df['event_type'] == 'Match'])}")
print(f"  • Training records: {len(master_df[master_df['event_type'] == 'Training'])}")

print("\n✅ PILLAR 3: Performance Prediction")
print(f"  • Match ratings generated: {rating_data['match_performance_rating'].notna().sum()}")
print(f"  • LSTM sequences created: {len(X_seq)}")
print(f"  • Sequence length: {SEQ_LENGTH} days")
if len(X_seq) >= 10:
    print(f"  • Model trained: YES")
    print(f"  • Test MAE: {test_mae:.4f}")
else:
    print(f"  • Model trained: NO (need more match data)")

print("\n📁 Output Files:")
print(f"  • {OUT_DIR / 'pillar1_calendar_j_days.csv'}")
print(f"  • {OUT_DIR / 'pillar2_master_features.csv'}")
if len(X_seq) >= 10:
    print(f"  • {OUT_DIR / 'lstm_performance_predictor.h5'}")

print("\n" + "="*60)
print("🚀 PIPELINE COMPLETE!")
print("="*60)

print("\n📝 Next Steps:")
print("  1. Review the generated files in data/processed/")
print("  2. Validate J-day calculations for specific players")
print("  3. Check feature distributions and missing values")
print("  4. If model trained, use it to predict upcoming match readiness")
print("  5. Integrate predictions into web dashboard")

print("\n💡 Usage Example:")
print("   - Load master_features.csv")
print("   - Filter for upcoming match (next J-Day)")
print("   - Extract last 10 days of training data")
print("   - Feed to LSTM model → Get Match Readiness Score")


🎯 CYCLE-TO-PEAK PIPELINE SUMMARY

✅ PILLAR 1: Training Cycle Calculation
  • Created calendar with J-day labels


NameError: name 'calendar' is not defined

In [3]:
! pip install keras2onnx

   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
    --------------------------------------- 0.3/16.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/16.5 MB 1.5 MB/s eta 0:00:11
   -- ------------------------------------- 1.0/16.5 MB 2.0 MB/s eta 0:00:08
   --- ------------------------------------ 1.3/16.5 MB 1.8 MB/s eta 0:00:09
   ---- ----------------------------------- 1.8/16.5 MB 1.9 MB/s eta 0:00:08
   ----- ---------------------------------- 2.1/16.5 MB 1.7 MB/s eta 0:00:09
   ----- ---------------------------------- 2.1/16.5 MB 1.7 MB/s eta 0:00:09
   ------ --------------------------------- 2.6/16.5 MB 1.6 MB/s eta 0:00:09
   ------- -------------------------------- 2.9/16.5 MB 1.6 MB/s eta 0:00:09
   ------- -------------------------------- 3.1/16.5 MB 1.6 MB/s eta 0:00:09
   -------- ------------------------------- 3.7/16.5 MB 1.6 MB/s eta 0:00:08
   --------- ------------------------------ 3.9/16.5 MB 1.6 MB/s eta 0:00:08
   ----------

# 📊 Visualisation des Résultats du Modèle

Cette section présente des visualisations complètes pour évaluer correctement les performances du modèle LSTM.

In [21]:
# ============================================================
# RECHARGEMENT DES DONNÉES POUR LES VISUALISATIONS
# ============================================================

import pandas as pd
import numpy as np
from pathlib import Path

# Dossiers de travail
OUT_DIR = Path("data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

calendar_path = OUT_DIR / "pillar1_calendar_j_days.csv"
master_path = OUT_DIR / "pillar2_master_features.csv"
model_path = OUT_DIR / "lstm_performance_predictor.h5"

if not calendar_path.exists() or not master_path.exists():
    print("❌ Fichiers prétraités introuvables dans data/processed. Exécutez d'abord le pipeline complet.")
else:
    # Charger les sorties pré-calculées
    calendar = pd.read_csv(calendar_path)
    master_df = pd.read_csv(master_path)

    # Harmoniser les dates
    for df in (calendar, master_df):
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'], errors='coerce')

    rating_col = 'match_performance_rating'

    def _recompute_match_ratings(df: pd.DataFrame) -> pd.DataFrame:
        """Recalcule la note de performance si elle est absente dans master_df."""
        print("ℹ️ Colonne 'match_performance_rating' absente. Recalcul en cours...")

        match_data = df[df['event_type'] == 'Match'].copy()
        if match_data.empty:
            print("⚠️ Aucun match disponible pour calculer les ratings.")
            return pd.DataFrame(columns=['player_name', 'team_name', 'date', rating_col])

        gps_rating_features = [
            'total_distance', 'high_speed_distance', 'sprint_distance',
            'total_player_load', 'high_accel_efforts', 'high_decel_efforts'
        ]
        wyscout_rating_features = [
            'goals', 'shots', 'xg', 'assists', 'successful_dribbles',
            'accurate_passes', 'total_passes', 'offensive_duels_won',
            'defensive_duels_won', 'interceptions'
        ]

        available_gps = [f for f in gps_rating_features if f in match_data.columns]
        available_wyscout = [f for f in wyscout_rating_features if f in match_data.columns]

        if not available_gps and not available_wyscout:
            print("⚠️ Impossible de recalculer les ratings: aucune feature pertinente disponible.")
            return pd.DataFrame(columns=['player_name', 'team_name', 'date', rating_col])

        required_subset = available_gps + available_wyscout[:min(3, len(available_wyscout))]
        rating_candidates = match_data.dropna(subset=required_subset) if required_subset else match_data

        if rating_candidates.empty:
            print("⚠️ Aucune ligne ne possède toutes les features nécessaires pour recalculer les ratings.")
            return pd.DataFrame(columns=['player_name', 'team_name', 'date', rating_col])

        all_features = available_gps + available_wyscout
        if not all_features:
            print("⚠️ Aucun ensemble complet de features pour le calcul des ratings.")
            return pd.DataFrame(columns=['player_name', 'team_name', 'date', rating_col])

        X = rating_candidates[all_features].fillna(0.0)

        try:
            from sklearn.preprocessing import StandardScaler
            from sklearn.decomposition import PCA

            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            n_components = min(5, X_scaled.shape[1], X_scaled.shape[0])
            if n_components < 1:
                raise ValueError("Nombre de composantes PCA insuffisant.")

            pca = PCA(n_components=n_components)
            pca.fit(X_scaled)
            weights = np.abs(pca.components_[0])
            if weights.sum() == 0:
                weights = np.ones_like(weights) / len(weights)
            else:
                weights = weights / weights.sum()

            weighted_scores = (X_scaled * weights).sum(axis=1)
        except Exception as exc:
            print(f"⚠️ PCA indisponible ({exc}). Utilisation d'une moyenne pondérée simple.")
            X_min = X.min(axis=0)
            X_max = X.max(axis=0)
            denom = (X_max - X_min).replace(0, 1)
            X_normalized = (X - X_min) / denom
            weighted_scores = X_normalized.mean(axis=1)

        min_score = float(weighted_scores.min())
        max_score = float(weighted_scores.max())
        if max_score == min_score:
            normalized_scores = np.full_like(weighted_scores, 5.0, dtype=float)
        else:
            normalized_scores = 1 + 9 * (weighted_scores - min_score) / (max_score - min_score)

        ratings_df = rating_candidates[['player_name', 'team_name', 'date']].copy()
        ratings_df[rating_col] = np.round(normalized_scores, 2)
        print(f"✅ Ratings recalculés pour {len(ratings_df)} matchs.")
        return ratings_df

    if rating_col not in master_df.columns:
        ratings_recomputed = _recompute_match_ratings(master_df)
        if not ratings_recomputed.empty:
            master_df = master_df.merge(
                ratings_recomputed,
                on=['player_name', 'team_name', 'date'],
                how='left'
            )
        else:
            master_df[rating_col] = np.nan

    # Reconstituer les notations de match (si disponibles)
    if rating_col in master_df.columns:
        rating_data = (
            master_df[
                (master_df['event_type'] == 'Match') &
                (master_df[rating_col].notna())
            ][['player_name', 'team_name', 'date', rating_col]]
            .drop_duplicates()
            .sort_values(['player_name', 'date'])
        )
    else:
        rating_data = pd.DataFrame(columns=['player_name', 'team_name', 'date', rating_col])

    # Ordonnancer l'ensemble principal
    master_df = master_df.sort_values(['player_name', 'date']).reset_index(drop=True)

    # Paramètres de séquences (identiques à l'entraînement)
    selected_features = [
        'total_distance',
        'high_speed_distance',
        'total_player_load',
        'high_accel_efforts',
        'is_match_day',
        'days_to_next_match',
    ]
    n_features = len(selected_features)
    SEQ_LENGTH = 10

    def _build_day_vector(row: pd.Series) -> list[float]:
        """Transforme une ligne en vecteur de features pour la séquence temporelle."""
        return [
            float(row.get('total_distance', 0) or 0),
            float(row.get('high_speed_distance', 0) or 0),
            float(row.get('total_player_load', 0) or 0),
            float(row.get('high_accel_efforts', 0) or 0),
            1.0 if row.get('event_type') == 'Match' else 0.0,
            float(row.get('days_to_next_match', 0) or 0),
        ]

    def recreate_sequences(df: pd.DataFrame, sequence_length: int = SEQ_LENGTH):
        """Recrée les séquences LSTM à partir du master_df sauvegardé."""
        sequences, targets, metadata = [], [], []

        if rating_col not in df.columns:
            return np.array([]), np.array([]), []

        for player, player_data in df.groupby('player_name'):
            player_data = player_data.sort_values('date').reset_index(drop=True)
            match_indices = player_data[
                (player_data['event_type'] == 'Match') &
                (player_data[rating_col].notna())
            ].index

            for match_idx in match_indices:
                target_row = player_data.iloc[match_idx]
                lookback_start = max(0, match_idx - sequence_length)
                history_window = player_data.iloc[lookback_start:match_idx]

                if history_window.empty:
                    continue

                seq_rows = [_build_day_vector(row) for _, row in history_window.iterrows()]

                while len(seq_rows) < sequence_length:
                    seq_rows.insert(0, [0.0] * n_features)

                sequences.append(seq_rows[:sequence_length])
                targets.append(float(target_row[rating_col]))
                metadata.append({
                    'player': player,
                    'team': target_row.get('team_name'),
                    'date': target_row['date'],
                })

        return np.array(sequences, dtype=float), np.array(targets, dtype=float), metadata

    X_seq, y_seq, seq_meta = recreate_sequences(master_df, sequence_length=SEQ_LENGTH)

    if len(X_seq) == 0:
        print("⚠️ Aucune séquence générée. Vérifiez que 'match_performance_rating' est disponible ou recalculée correctement.")
    else:
        print(f"✅ Séquences rechargées: {len(X_seq)} | Forme: {X_seq.shape}")
        print(f"   • Cible: {y_seq.shape}")

        if len(X_seq) < 10:
            print("⚠️ Peu de séquences disponibles (<10). Les visualisations pourront être limitées.")
        else:
            from sklearn.model_selection import train_test_split
            from sklearn.preprocessing import MinMaxScaler

            X_train, X_test, y_train, y_test = train_test_split(
                X_seq, y_seq, test_size=0.2, random_state=42
            )

            scaler_X = MinMaxScaler()
            n_steps = X_train.shape[1]

            X_train_scaled = scaler_X.fit_transform(X_train.reshape(-1, n_features)).reshape(-1, n_steps, n_features)
            X_test_scaled = scaler_X.transform(X_test.reshape(-1, n_features)).reshape(-1, n_steps, n_features)

            print(f"✅ Jeux recréés: {X_train.shape[0]} entraînement | {X_test.shape[0]} test")
            print(f"   • Nombre de features: {n_features}")

    print("\nLes variables clés (calendar, master_df, rating_data, X_seq, y_seq, X_train_scaled, X_test_scaled, etc.) sont prêtes pour les visualisations.")

ℹ️ Colonne 'match_performance_rating' absente. Recalcul en cours...
✅ Ratings recalculés pour 2536 matchs.
✅ Ratings recalculés pour 2536 matchs.
✅ Séquences rechargées: 212317 | Forme: (212317, 10, 6)
   • Cible: (212317,)
✅ Séquences rechargées: 212317 | Forme: (212317, 10, 6)
   • Cible: (212317,)
✅ Jeux recréés: 169853 entraînement | 42464 test
   • Nombre de features: 6

Les variables clés (calendar, master_df, rating_data, X_seq, y_seq, X_train_scaled, X_test_scaled, etc.) sont prêtes pour les visualisations.
✅ Jeux recréés: 169853 entraînement | 42464 test
   • Nombre de features: 6

Les variables clés (calendar, master_df, rating_data, X_seq, y_seq, X_train_scaled, X_test_scaled, etc.) sont prêtes pour les visualisations.


In [22]:
# ============================================================
# CHARGEMENT DU MODÈLE SAUVEGARDÉ
# ============================================================

import os
from pathlib import Path
import numpy as np

# Vérifier si le modèle existe dans data/processed
model_path = OUT_DIR / "lstm_performance_predictor.h5"

if os.path.exists(model_path):
    print(f"📂 Chargement du modèle depuis: {model_path}")
    
    try:
        # Import TensorFlow et Keras
        import tensorflow as tf
        from tensorflow import keras
        
        # Gestion des pertes/metrics sérialisés en texte
        custom_objects = {
            'mse': keras.losses.MeanSquaredError(),
            'mae': keras.metrics.MeanAbsoluteError()
        }
        
        try:
            # Charger le modèle (compile=True) avec custom_objects pour éviter l'erreur "Could not locate function"
            model = keras.models.load_model(str(model_path), custom_objects=custom_objects)
        except TypeError as te:
            if "Could not locate function" in str(te):
                print("⚠️ Impossible de localiser certaines fonctions sérialisées (mse/mae). Chargement avec compile=False...")
                model = keras.models.load_model(str(model_path), custom_objects=custom_objects, compile=False)
                model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=[keras.metrics.MeanAbsoluteError()])
                print("✅ Modèle rechargé et recompilé avec succès!")
            else:
                raise
        
        print("✅ Modèle chargé avec succès!")
        print("\n📋 Architecture du modèle:")
        model.summary()
        
        # Vérifier que les variables nécessaires existent pour les visualisations
        if 'X_test_scaled' not in globals() or 'y_test' not in globals():
            print("\n⚠️ Note: Les données de test (X_test_scaled, y_test) ne sont pas encore chargées.")
            print("   Vous devez exécuter les cellules précédentes pour générer ces variables.")
        else:
            # Évaluer le modèle pour retrouver les métriques de test
            test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=0)
            
            # Régénérer les prédictions avec le modèle chargé
            print("\n🔄 Régénération des prédictions avec le modèle chargé...")
            y_pred = model.predict(X_test_scaled, verbose=0).flatten()
            residuals = y_test - y_pred
            abs_errors = np.abs(residuals)
            
            # Calculer les métriques complémentaires
            from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)
            mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
            
            # Calculer les quartiles pour les visualisations
            quartiles = np.percentile(abs_errors, [25, 50, 75])
            q1_count = np.sum(abs_errors <= quartiles[0])
            q2_count = np.sum((abs_errors > quartiles[0]) & (abs_errors <= quartiles[1]))
            q3_count = np.sum((abs_errors > quartiles[1]) & (abs_errors <= quartiles[2]))
            q4_count = np.sum(abs_errors > quartiles[2])
            quartile_counts = [q1_count, q2_count, q3_count, q4_count]
            
            # Calculer les précisions par seuil
            accuracy_within_05 = np.sum(abs_errors < 0.5) / len(abs_errors) * 100
            accuracy_within_1 = np.sum(abs_errors < 1.0) / len(abs_errors) * 100
            
            print(f"✅ Prédictions générées: {len(y_pred)} échantillons")
            print("\n📊 Métriques calculées:")
            print(f"   • MSE (Keras): {test_loss:.4f}")
            print(f"   • MAE (Keras): {test_mae:.4f}")
            print(f"   • MAE (sklearn): {mae:.4f}")
            print(f"   • RMSE: {rmse:.4f}")
            print(f"   • R²: {r2:.4f}")
            print(f"   • MAPE: {mape:.2f}%")
            print("\n📊 Aperçu des prédictions:")
            for i in range(min(5, len(y_test))):
                print(f"  Match {i+1}: Réel={y_test[i]:.2f} | Prédit={y_pred[i]:.2f} | Erreur={abs_errors[i]:.2f}")
                
    except ImportError as ie:
        print(f"❌ Erreur d'import: {ie}")
        print("   Installez TensorFlow avec: pip install tensorflow")
    except Exception as e:
        print(f"❌ Erreur lors du chargement du modèle: {e}")
        print(f"   Type d'erreur: {type(e).__name__}")
else:
    print(f"❌ Le fichier modèle n'existe pas: {model_path}")

📂 Chargement du modèle depuis: data\processed\lstm_performance_predictor.h5
✅ Modèle chargé avec succès!

📋 Architecture du modèle:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10, 64)         │        18,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,139 (121.64 KB)

 Trainable params: 31,137 (121.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)


🔄 Régénération des prédictions avec le modèle chargé...
✅ Prédictions générées: 42464 échantillons

📊 Métriques calculées:
   • MSE (Keras): 0.7868
   • MAE (Keras): 0.6668
   • MAE (sklearn): 0.6668
   • RMSE: 0.8870
   • R²: 0.1321
   • MAPE: 31.81%

📊 Aperçu des prédictions:
  Match 1: Réel=2.04 | Prédit=2.12 | Erreur=0.08
  Match 2: Réel=2.73 | Prédit=2.16 | Erreur=0.57
  Match 3: Réel=1.31 | Prédit=2.16 | Erreur=0.85
  Match 4: Réel=1.53 | Prédit=2.31 | Erreur=0.78
  Match 5: Réel=1.24 | Prédit=2.14 | Erreur=0.90
✅ Prédictions générées: 42464 échantillons

📊 Métriques calculées:
   • MSE (Keras): 0.7868
   • MAE (Keras): 0.6668
   • MAE (sklearn): 0.6668
   • RMSE: 0.8870
   • R²: 0.1321
   • MAPE: 31.81%

📊 Aperçu des prédictions:
  Match 1: Réel=2.04 | Prédit=2.12 | Erreur=0.08
  Match 2: Réel=2.73 | Prédit=2.16 | Erreur=0.57
  Match 3: Réel=1.31 | Prédit=2.16 | Erreur=0.85
  Match 4: Réel=1.53 | Prédit=2.31 | Erreur=0.78
  Match 5: Réel=1.24 | Prédit=2.14 | Erreur=0.90
